# Tiling

### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) Georeferenzierung fallen lassen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen 

In [15]:
import os
import sys
from PIL import Image


savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

img = Image.open(filename)
#tiling(filename, savedir, (256, 256))

UnidentifiedImageError: cannot identify image file 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_1.tiff'

In [12]:
# define tiling 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            save_to= os.path.join(savedir, "testing_{row}_{col}.jpg".format(row=row_i, col=col_i))
            crop.save(save_to)


import os
import sys
from PIL import Image

savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_1.tiff"

img = Image.open(filename)
width, height = img.size
start_pos = start_x, start_y = (0, 0)
cropped_image_size = w, h = (256,256)

frame_num = 1
for col_i in range(0, width, w):
    for row_i in range(0, height, h):
        crop = img.crop((col_i, row_i, col_i + w, row_i + h))
        save_to= os.path.join(savedir, "testing_{:02}.png")
        crop.save(save_to.format(frame_num))
        frame_num += 1

### Andere Methode, hier fehlen Randpieces! (nur 270 tiles, bei oberem mehr tiles, auch Randtiles)

In [17]:
tile('img.jpg', 'E:/W_Katrin/InnovationLab', 'E:/W_Katrin/InnovationLab/test', 256)

### This method results in a list of arrays

In [20]:
im = mpimg.imread('img.jpg')
M = 256
N=256
tiles = [im[x:x+M,y:y+N] for x in range(0,im.shape[0],M) for y in range(0,im.shape[1],N)]


## TODO

develop code to enter many images and return tiles in a logical way (also think of mask pixels!!) 

Problem: we have images, but we need the mask to be tiled at the same location the exact same way!